In [44]:
import polars as pl
import pandas as pd
import glob
import os

from sklearn.preprocessing import LabelEncoder

In [15]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
        
        if col[-1] == 'D':
            df = df.with_columns(pl.col(col).str.to_date())

    return df

In [16]:
df = pl.read_parquet('../data/raw/parquet_files/train/train_credit_bureau_a_1_0.parquet').pipe((set_table_dtypes))

In [17]:
df

case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,lastupdate_1112D,lastupdate_388D,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,…,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax2date_1002D,overdueamountmax2date_1142D,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
i64,f64,f64,str,str,str,str,f64,f64,f64,date,date,date,date,date,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,date,date,f64,f64,f64,f64,i64,f64,…,f64,f64,date,date,f64,f64,f64,f64,f64,f64,f64,f64,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,date,f64,f64,str,str,f64,f64,f64,f64,f64,f64
388,null,null,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""",null,null,null,2023-06-20,null,null,2018-06-20,null,374419.5,0.0,"""a55475b1""",0.0,null,null,7.0,2018.0,null,"""a55475b1""","""55b002a9""",null,null,null,2019-01-24,null,7811.4463,null,null,null,0,2.0,…,0.0,null,null,null,0.0,null,null,260093.7,null,0.0,0.0,null,null,null,0.0,null,null,7.0,2018.0,null,null,30.0,null,null,"""96a8fdfe""","""a55475b1""",null,null,null,"""ab3c25cf""","""ab3c25cf""",null,268897.62,0.0,0.0,374419.5,0.0
388,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""",null,null,135806.0,2020-08-06,null,null,2018-08-06,null,null,null,"""a55475b1""",0.0,null,null,8.0,2018.0,null,"""a55475b1""","""P204_66_73""",8742.8,null,null,2019-01-11,null,8742.8,null,null,null,1,null,…,0.0,null,null,null,0.0,null,null,null,null,0.0,0.0,null,null,null,0.0,null,null,8.0,2018.0,null,null,null,null,null,"""60c73645""","""a55475b1""",null,null,114325.805,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,2,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,3,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,n

In [201]:
date_cols = [ df.columns[i] for i in range(len(df.columns)) if df.dtypes[i] == pl.Date ]

# Categorical columns
cat_cols = [ df.columns[i] for i in range(len(df.columns)) if df.dtypes[i] == pl.String ]

ignore_cols = ['case_id', 'num_group1', 'num_group2']
num_cols = [ 
    df.columns[i] for i in range(len(df.columns)) 
    if (df.columns[i] not in date_cols) and (df.columns[i] not in cat_cols) and (df.columns[i] not in ignore_cols)
]

In [103]:
def separate_dates(df: pl.DataFrame, date_cols: list[str] = []) -> pl.DataFrame:

    date_df = df.select(date_cols)

    for col in date_cols:
        date_df = date_df.with_columns(pl.col(col).dt.year().alias(col + '_year'))
        date_df = date_df.with_columns(pl.col(col).dt.month().alias(col + '_month'))
        date_df = date_df.with_columns(pl.col(col).dt.day().alias(col + '_day'))
        date_df.drop_in_place(col)

    return date_df

In [109]:
date_df = separate_dates(df, date_cols=date_cols)

In [110]:
date_df.head()

dateofcredend_289D_year,dateofcredend_289D_month,dateofcredend_289D_day,dateofcredend_353D_year,dateofcredend_353D_month,dateofcredend_353D_day,dateofcredstart_181D_year,dateofcredstart_181D_month,dateofcredstart_181D_day,dateofcredstart_739D_year,dateofcredstart_739D_month,dateofcredstart_739D_day,dateofrealrepmt_138D_year,dateofrealrepmt_138D_month,dateofrealrepmt_138D_day,lastupdate_1112D_year,lastupdate_1112D_month,lastupdate_1112D_day,lastupdate_388D_year,lastupdate_388D_month,lastupdate_388D_day,numberofoverdueinstlmaxdat_148D_year,numberofoverdueinstlmaxdat_148D_month,numberofoverdueinstlmaxdat_148D_day,numberofoverdueinstlmaxdat_641D_year,numberofoverdueinstlmaxdat_641D_month,numberofoverdueinstlmaxdat_641D_day,overdueamountmax2date_1002D_year,overdueamountmax2date_1002D_month,overdueamountmax2date_1002D_day,overdueamountmax2date_1142D_year,overdueamountmax2date_1142D_month,overdueamountmax2date_1142D_day,refreshdate_3813885D_year,refreshdate_3813885D_month,refreshdate_3813885D_day
i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8
2023,6,20,null,null,null,null,null,null,2018,6,20,null,null,null,2019,1,24,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,8,6,null,null,null,null,null,null,2018,8,6,null,null,null,2019,1,11,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,28
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,28
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,28


In [119]:
new_df = pl.concat([df, date_df], how='horizontal')

In [120]:
def freq_encoder(df: pl.DataFrame, cat_cols: list[str] = []) -> pl.DataFrame:

    cat_df = df.select(cat_cols)

    for col in cat_cols:
        # Calculate frequency for each category in the column
        value_counts = cat_df.groupby(col).agg(pl.len().alias('count'))
        total_count = cat_df.height  # Use height for row count in Polars
        frequency = (value_counts.with_columns(
                        (value_counts['count'] / total_count).alias(f'{col}_freq')
                    ).select([col, f'{col}_freq']))
        
        # Joining the frequency DataFrame back to the original DataFrame
        cat_df = cat_df.join(frequency, on=col, how='left')
        cat_df.drop_in_place(col)

    return cat_df

In [125]:
freq_df = freq_encoder(df, cat_cols=cat_cols)

/var/folders/sp/dv4f3qgx20vb8f6ghm2mh5880000gn/T/ipykernel_64627/3365233391.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  value_counts = cat_df.groupby(col).agg(pl.len().alias('count'))


In [126]:
freq_df.head()

classificationofcontr_13M_freq,classificationofcontr_400M_freq,contractst_545M_freq,contractst_964M_freq,description_351M_freq,financialinstitution_382M_freq,financialinstitution_591M_freq,purposeofcred_426M_freq,purposeofcred_874M_freq,subjectrole_182M_freq,subjectrole_93M_freq
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.002313,0.889024,0.154702,0.888859,0.995765,0.888827,0.001622,0.035934,0.888839,0.078808,0.070128
0.139587,0.889024,0.154702,0.888859,0.995765,0.888827,0.061924,0.122771,0.888839,0.917829,0.927316
0.839874,0.889024,0.840134,0.888859,0.995765,0.888827,0.839736,0.839736,0.888839,0.917829,0.927316
0.839874,0.889024,0.840134,0.888859,0.995765,0.888827,0.839736,0.839736,0.888839,0.917829,0.927316
0.839874,0.889024,0.840134,0.888859,0.995765,0.888827,0.839736,0.839736,0.888839,0.917829,0.927316


In [127]:
def binary_encoder(df: pl.DataFrame, cat_cols: list[str] = []) -> pl.DataFrame:
    # Initialize LabelEncoder
    le = LabelEncoder()

    cat_df = df.select(cat_cols)

    # Binary encoding for each categorical column
    for col in cat_cols:
        # Convert categories to integers using LabelEncoder from sklearn
        encoded_int = le.fit_transform(cat_df[col].to_numpy())

        # Convert the numpy array back to a Polars Series and rename it
        int_col_name = f"{col}_int"
        encoded_series = pl.Series(encoded_int).alias(int_col_name)

        # Add the integer encoded column to the DataFrame
        cat_df = cat_df.with_columns(encoded_series)

        # Calculate the maximum binary length
        max_binary_length = encoded_series.max().bit_length()

        # Create binary encoding directly
        for bit_position in range(max_binary_length):
            # Use bitwise operations directly within Polars
            bit_value = (encoded_series / (2 ** bit_position)).cast(pl.Int64) & 1
            new_col_name = f"{col}_binary_{bit_position}"
            cat_df = cat_df.with_columns(
                bit_value.alias(new_col_name)
            )
            cat_df = cat_df.with_columns(
                pl.col(new_col_name).cast(pl.Int8)
            )

        cat_df.drop_in_place(int_col_name)
        cat_df.drop_in_place(col)
    
    return cat_df

In [129]:
bin_df = binary_encoder(df, cat_cols)

In [131]:
final_df = pl.concat([df, date_df, freq_df, bin_df], how='horizontal')

In [133]:
final_df = final_df.drop(date_cols + cat_cols)

In [134]:
final_df

case_id,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,…,contractst_964M_binary_6,contractst_964M_binary_7,description_351M_binary_0,description_351M_binary_1,description_351M_binary_2,description_351M_binary_3,financialinstitution_382M_binary_0,financialinstitution_382M_binary_1,financialinstitution_382M_binary_2,financialinstitution_382M_binary_3,financialinstitution_382M_binary_4,financialinstitution_382M_binary_5,financialinstitution_382M_binary_6,financialinstitution_382M_binary_7,financialinstitution_591M_binary_0,financialinstitution_591M_binary_1,financialinstitution_591M_binary_2,financialinstitution_591M_binary_3,financialinstitution_591M_binary_4,financialinstitution_591M_binary_5,financialinstitution_591M_binary_6,financialinstitution_591M_binary_7,purposeofcred_426M_binary_0,purposeofcred_426M_binary_1,purposeofcred_426M_binary_2,purposeofcred_426M_binary_3,purposeofcred_874M_binary_0,purposeofcred_874M_binary_1,purposeofcred_874M_binary_2,purposeofcred_874M_binary_3,purposeofcred_874M_binary_4,subjectrole_182M_binary_0,subjectrole_182M_binary_1,subjectrole_182M_binary_2,subjectrole_93M_binary_0,subjectrole_93M_binary_1,subjectrole_93M_binary_2
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
388,null,null,null,null,null,374419.5,0.0,0.0,null,null,7.0,2018.0,null,null,null,null,7811.4463,null,null,null,0,2.0,1.0,null,60.0,null,54.0,0.0,null,0.0,null,null,260093.7,null,0.0,0.0,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,1
388,null,null,null,null,135806.0,null,null,0.0,null,null,8.0,2018.0,null,8742.8,null,null,8742.8,null,null,null,1,null,null,null,null,null,null,0.0,null,0.0,null,null,null,null,0.0,0.0,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0
388,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0
388,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0
388,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2588481,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0
2588481,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7,null,null,null,null,null,null,null,null,null,null,

In [135]:
df

case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,lastupdate_1112D,lastupdate_388D,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,…,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax2date_1002D,overdueamountmax2date_1142D,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
i64,f64,f64,str,str,str,str,f64,f64,f64,date,date,date,date,date,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,date,date,f64,f64,f64,f64,i64,f64,…,f64,f64,date,date,f64,f64,f64,f64,f64,f64,f64,f64,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,date,f64,f64,str,str,f64,f64,f64,f64,f64,f64
388,null,null,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""",null,null,null,2023-06-20,null,null,2018-06-20,null,374419.5,0.0,"""a55475b1""",0.0,null,null,7.0,2018.0,null,"""a55475b1""","""55b002a9""",null,null,null,2019-01-24,null,7811.4463,null,null,null,0,2.0,…,0.0,null,null,null,0.0,null,null,260093.7,null,0.0,0.0,null,null,null,0.0,null,null,7.0,2018.0,null,null,30.0,null,null,"""96a8fdfe""","""a55475b1""",null,null,null,"""ab3c25cf""","""ab3c25cf""",null,268897.62,0.0,0.0,374419.5,0.0
388,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""",null,null,135806.0,2020-08-06,null,null,2018-08-06,null,null,null,"""a55475b1""",0.0,null,null,8.0,2018.0,null,"""a55475b1""","""P204_66_73""",8742.8,null,null,2019-01-11,null,8742.8,null,null,null,1,null,…,0.0,null,null,null,0.0,null,null,null,null,0.0,0.0,null,null,null,0.0,null,null,8.0,2018.0,null,null,null,null,null,"""60c73645""","""a55475b1""",null,null,114325.805,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,2,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,3,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,n

In [179]:
cat_aggs = [ pl.col(col).mode() for col in cat_cols ]
df_cat_grouped = df.group_by('case_id').agg(cat_aggs)

In [180]:
df_cat_grouped.head()

case_id,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
i64,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
1406101,"[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""7241344e""]","[""a55475b1""]","[""P150_136_157"", ""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]"
719402,"[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]"
2581504,"[""a55475b1""]","[""ea6782cc""]","[""a55475b1""]","[""7241344e""]","[""a55475b1""]","[""Home Credit""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]"
2560922,"[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]"
663536,"[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]"


In [181]:
for col in df_cat_grouped.columns:
    if df_cat_grouped[col].dtype != pl.Int64:
        df_cat_grouped = df_cat_grouped.with_columns(pl.col(col).map_elements(lambda x: x[0], return_dtype=pl.String))


In [182]:
df_cat_grouped

case_id,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
i64,str,str,str,str,str,str,str,str,str,str,str
1406101,"""a55475b1""","""a55475b1""","""a55475b1""","""7241344e""","""a55475b1""","""P150_136_157""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
719402,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
2581504,"""a55475b1""","""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""","""Home Credit""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
2560922,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
663536,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
…,…,…,…,…,…,…,…,…,…,…,…
1417465,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
104574,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
136309,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [217]:
def group_file_data(
    df: pl.DataFrame, 
    num_cols: list[str] = [], 
    date_cols: list[str] = [], 
    cat_cols: list[str] = []
) -> pl.DataFrame:
    '''
    Function to group numerical, date, and categorical columns

    Parameters:
    -----------
    df : Polars DataFrame
    num_cols : List of numerical column names (remember to drop num_group columns)
    date_cols : List of date column names
    cat_cols : List of categorical column names
    '''
    
    # Convert date columns
    df_date = df[['case_id'] + date_cols]

    # One-hot categories
    # df_dummies = df[['case_id'] + cat_cols].to_dummies(cat_cols)
    df_cat = df[['case_id'] + cat_cols]

    # Num DataFrame
    df_num = df[['case_id'] + num_cols]

    # Date aggs
    date_aggs = [ pl.min(col).name.suffix('_min') for col in date_cols ] +\
                [ pl.max(col).name.suffix('_max') for col in date_cols ] +\
                [ pl.n_unique(col).name.suffix('_distinct') for col in date_cols]
    df_date_grouped = df_date.group_by('case_id').agg(date_aggs)

    # One-hot aggs
    # dummy_cols = [ col for col in df_dummies.columns if col != 'case_id']
    # dummies_aggs = [ pl.sum(col).name.suffix('_sum') for col in dummy_cols ]
    # df_dummies_grouped = df_dummies.group_by('case_id').agg(dummies_aggs)

    # Cat aggs
    # cat_aggs = [ pl.col(col).mode().name.suffix('_mode') for col in cat_cols ] +\
            #    [ pl.n_unique(col).name.suffix('_distinct') for col in cat_cols ]
    cat_aggs = [ pl.col(col).mode() for col in cat_cols ]
    df_cat_grouped = df_cat.group_by('case_id').agg(cat_aggs)
    for col in df_cat_grouped.columns:
        if df_cat_grouped[col].dtype != pl.Int64:
            df_cat_grouped = df_cat_grouped.with_columns(pl.col(col).map_elements(lambda x: x[0], return_dtype=pl.String))

    # Numerical aggs
    num_aggs = [ pl.min(col).name.suffix('_min') for col in num_cols ] +\
            [ pl.max(col).name.suffix('_max') for col in num_cols ] +\
            [ pl.mean(col).name.suffix('_mean') for col in num_cols ] +\
            [ pl.median(col).name.suffix('_median') for col in num_cols ] +\
            [ pl.sum(col).name.suffix('_sum') for col in num_cols ]
    df_num_grouped = df_num.group_by('case_id').agg(num_aggs)

    # Join DataFrames
    df_joined = df_num_grouped.join(df_date_grouped, on='case_id')
    # df_joined = df_joined.join(df_dummies_grouped, on='case_id')
    df_joined = df_joined.join(df_cat_grouped, on='case_id')

    return df_joined

In [218]:
grouped_df = group_file_data(df, num_cols=num_cols, date_cols=date_cols, cat_cols=cat_cols)

In [219]:
grouped_df.filter(pl.col('case_id') == 388)

case_id,annualeffectiverate_199L_min,annualeffectiverate_63L_min,contractsum_5085717L_min,credlmt_230A_min,credlmt_935A_min,debtoutstand_525A_min,debtoverdue_47A_min,dpdmax_139P_min,dpdmax_757P_min,dpdmaxdatemonth_442T_min,dpdmaxdatemonth_89T_min,dpdmaxdateyear_596T_min,dpdmaxdateyear_896T_min,instlamount_768A_min,instlamount_852A_min,interestrate_508L_min,monthlyinstlamount_332A_min,monthlyinstlamount_674A_min,nominalrate_281L_min,nominalrate_498L_min,numberofcontrsvalue_258L_min,numberofcontrsvalue_358L_min,numberofinstls_229L_min,numberofinstls_320L_min,numberofoutstandinstls_520L_min,numberofoutstandinstls_59L_min,numberofoverdueinstlmax_1039L_min,numberofoverdueinstlmax_1151L_min,numberofoverdueinstls_725L_min,numberofoverdueinstls_834L_min,outstandingamount_354A_min,outstandingamount_362A_min,overdueamount_31A_min,overdueamount_659A_min,overdueamountmax2_14A_min,overdueamountmax2_398A_min,…,overdueamountmax2date_1142D_min,refreshdate_3813885D_min,dateofcredend_289D_max,dateofcredend_353D_max,dateofcredstart_181D_max,dateofcredstart_739D_max,dateofrealrepmt_138D_max,lastupdate_1112D_max,lastupdate_388D_max,numberofoverdueinstlmaxdat_148D_max,numberofoverdueinstlmaxdat_641D_max,overdueamountmax2date_1002D_max,overdueamountmax2date_1142D_max,refreshdate_3813885D_max,dateofcredend_289D_distinct,dateofcredend_353D_distinct,dateofcredstart_181D_distinct,dateofcredstart_739D_distinct,dateofrealrepmt_138D_distinct,lastupdate_1112D_distinct,lastupdate_388D_distinct,numberofoverdueinstlmaxdat_148D_distinct,numberofoverdueinstlmaxdat_641D_distinct,overdueamountmax2date_1002D_distinct,overdueamountmax2date_1142D_distinct,refreshdate_3813885D_distinct,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,date,date,date,date,date,date,date,date,date,date,date,date,date,date,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,str,str,str,str,str,str,str,str,str,str,str
388,null,null,null,null,135806.0,374419.5,0.0,0.0,null,null,7.0,2018.0,null,8742.8,null,null,7811.4463,null,null,null,2.0,1.0,null,60.0,null,54.0,0.0,null,0.0,null,null,260093.7,null,0.0,0.0,null,…,null,2018-11-02,2023-06-20,null,null,2018-08-06,null,2019-01-24,null,null,null,null,null,2019-01-28,3,1,1,3,1,3,1,1,1,1,1,7,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [196]:
new_date_cols = [ col for col in grouped_df.columns if grouped_df[col].dtype == pl.Date ]

In [198]:
date_df = separate_dates(grouped_df, new_date_cols)

In [199]:
date_df

dateofcredend_289D_min_year,dateofcredend_289D_min_month,dateofcredend_289D_min_day,dateofcredend_353D_min_year,dateofcredend_353D_min_month,dateofcredend_353D_min_day,dateofcredstart_181D_min_year,dateofcredstart_181D_min_month,dateofcredstart_181D_min_day,dateofcredstart_739D_min_year,dateofcredstart_739D_min_month,dateofcredstart_739D_min_day,dateofrealrepmt_138D_min_year,dateofrealrepmt_138D_min_month,dateofrealrepmt_138D_min_day,lastupdate_1112D_min_year,lastupdate_1112D_min_month,lastupdate_1112D_min_day,lastupdate_388D_min_year,lastupdate_388D_min_month,lastupdate_388D_min_day,numberofoverdueinstlmaxdat_148D_min_year,numberofoverdueinstlmaxdat_148D_min_month,numberofoverdueinstlmaxdat_148D_min_day,numberofoverdueinstlmaxdat_641D_min_year,numberofoverdueinstlmaxdat_641D_min_month,numberofoverdueinstlmaxdat_641D_min_day,overdueamountmax2date_1002D_min_year,overdueamountmax2date_1002D_min_month,overdueamountmax2date_1002D_min_day,overdueamountmax2date_1142D_min_year,overdueamountmax2date_1142D_min_month,overdueamountmax2date_1142D_min_day,refreshdate_3813885D_min_year,refreshdate_3813885D_min_month,refreshdate_3813885D_min_day,dateofcredend_289D_max_year,dateofcredend_289D_max_month,dateofcredend_289D_max_day,dateofcredend_353D_max_year,dateofcredend_353D_max_month,dateofcredend_353D_max_day,dateofcredstart_181D_max_year,dateofcredstart_181D_max_month,dateofcredstart_181D_max_day,dateofcredstart_739D_max_year,dateofcredstart_739D_max_month,dateofcredstart_739D_max_day,dateofrealrepmt_138D_max_year,dateofrealrepmt_138D_max_month,dateofrealrepmt_138D_max_day,lastupdate_1112D_max_year,lastupdate_1112D_max_month,lastupdate_1112D_max_day,lastupdate_388D_max_year,lastupdate_388D_max_month,lastupdate_388D_max_day,numberofoverdueinstlmaxdat_148D_max_year,numberofoverdueinstlmaxdat_148D_max_month,numberofoverdueinstlmaxdat_148D_max_day,numberofoverdueinstlmaxdat_641D_max_year,numberofoverdueinstlmaxdat_641D_max_month,numberofoverdueinstlmaxdat_641D_max_day,overdueamountmax2date_1002D_max_year,overdueamountmax2date_1002D_max_month,overdueamountmax2date_1002D_max_day,overdueamountmax2date_1142D_max_year,overdueamountmax2date_1142D_max_month,overdueamountmax2date_1142D_max_day,refreshdate_3813885D_max_year,refreshdate_3813885D_max_month,refreshdate_3813885D_max_day
i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8,i32,i8,i8
2019,7,19,2016,9,27,2016,6,27,2014,8,27,2016,9,28,2019,6,7,2016,9,28,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,3,2020,8,27,2018,11,19,2018,8,19,2019,1,16,2018,11,20,2019,6,12,2018,11,20,null,null,null,null,null,null,null,null,null,null,null,null,2019,6,22
2020,9,17,null,null,null,null,null,null,2018,9,17,null,null,null,2019,2,12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,3,2020,9,17,null,null,null,null,null,null,2018,9,17,null,null,null,2019,2,12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,2,27
2020,5,1,null,null,null,null,null,null,2018,10,5,null,null,null,2019,4,23,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,3,2020,10,5,null,null,null,null,null,null,2018,12,31,null,null,null,2019,4,28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,5,3
2019,10,29,null,null,null,null,null,null,2017,10,29,null,null,null,2019,6,18,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,1,3,2020,5,29,null,null,null,null,null,null,2019,3,18,null,null,null,2019,6,28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019,7,1
2021,7,4,2008,8,10,2007,5,3,2019,1,3,2008,9,8,2019,6,10,2008,9,15,2008,8,15,null,null,null,2008,8,15,null,null,null,2019,1,3,2021,7,4,2019,6,2,2017,12,3,2019,1,3,2019,6,3,2019,6,10,2019,6,3,2018,9,27,null,null,null,2018,9,27

In [224]:
df_num_group_0 = df.filter(pl.col('num_group1') == 0)
df_num_group_rest = df.filter(pl.col('num_group1') != 0)

df_num_group_0 = group_file_data(df_num_group_0, num_cols, date_cols, cat_cols)
df_num_group_rest = group_file_data(df_num_group_rest, num_cols, date_cols, cat_cols)

new_date_cols = [ 
    col for col in df_num_group_0.columns 
    if df_num_group_0[col].dtype == pl.Date 
]

# Transform data
date_df_0 = separate_dates(df_num_group_0, new_date_cols)
date_df_rest = separate_dates(df_num_group_rest, new_date_cols)
freq_df_0 = freq_encoder(df_num_group_0, cat_cols)
freq_df_rest = freq_encoder(df_num_group_rest, cat_cols)
bin_df_0 = binary_encoder(df_num_group_0, cat_cols)
bin_df_rest = binary_encoder(df_num_group_rest, cat_cols)

# Merge data
df_0 = pl.concat([df_num_group_0, date_df_0, freq_df_0, bin_df_0], how='horizontal')
df_rest = pl.concat([df_num_group_rest, date_df_rest, freq_df_rest, bin_df_rest], how='horizontal')

# Drop unprocessed columns
df_0 = df_0.drop(date_cols + cat_cols)
df_rest = df_rest.drop(date_cols + cat_cols)

/var/folders/sp/dv4f3qgx20vb8f6ghm2mh5880000gn/T/ipykernel_64627/3365233391.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  value_counts = cat_df.groupby(col).agg(pl.len().alias('count'))


In [225]:
df_0

case_id,annualeffectiverate_199L_min,annualeffectiverate_63L_min,contractsum_5085717L_min,credlmt_230A_min,credlmt_935A_min,debtoutstand_525A_min,debtoverdue_47A_min,dpdmax_139P_min,dpdmax_757P_min,dpdmaxdatemonth_442T_min,dpdmaxdatemonth_89T_min,dpdmaxdateyear_596T_min,dpdmaxdateyear_896T_min,instlamount_768A_min,instlamount_852A_min,interestrate_508L_min,monthlyinstlamount_332A_min,monthlyinstlamount_674A_min,nominalrate_281L_min,nominalrate_498L_min,numberofcontrsvalue_258L_min,numberofcontrsvalue_358L_min,numberofinstls_229L_min,numberofinstls_320L_min,numberofoutstandinstls_520L_min,numberofoutstandinstls_59L_min,numberofoverdueinstlmax_1039L_min,numberofoverdueinstlmax_1151L_min,numberofoverdueinstls_725L_min,numberofoverdueinstls_834L_min,outstandingamount_354A_min,outstandingamount_362A_min,overdueamount_31A_min,overdueamount_659A_min,overdueamountmax2_14A_min,overdueamountmax2_398A_min,…,contractst_964M_binary_4,contractst_964M_binary_5,contractst_964M_binary_6,description_351M_binary_0,description_351M_binary_1,description_351M_binary_2,description_351M_binary_3,financialinstitution_382M_binary_0,financialinstitution_382M_binary_1,financialinstitution_382M_binary_2,financialinstitution_382M_binary_3,financialinstitution_382M_binary_4,financialinstitution_382M_binary_5,financialinstitution_382M_binary_6,financialinstitution_382M_binary_7,financialinstitution_591M_binary_0,financialinstitution_591M_binary_1,financialinstitution_591M_binary_2,financialinstitution_591M_binary_3,financialinstitution_591M_binary_4,financialinstitution_591M_binary_5,financialinstitution_591M_binary_6,purposeofcred_426M_binary_0,purposeofcred_426M_binary_1,purposeofcred_426M_binary_2,purposeofcred_426M_binary_3,purposeofcred_874M_binary_0,purposeofcred_874M_binary_1,purposeofcred_874M_binary_2,purposeofcred_874M_binary_3,purposeofcred_874M_binary_4,subjectrole_182M_binary_0,subjectrole_182M_binary_1,subjectrole_182M_binary_2,subjectrole_93M_binary_0,subjectrole_93M_binary_1,subjectrole_93M_binary_2
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
1276735,null,null,null,null,0.0,371784.72,0.0,0.0,null,null,3.0,2017.0,null,0.0,null,null,0.0,null,null,null,4.0,2.0,null,null,null,null,0.0,null,0.0,null,null,null,null,0.0,0.0,null,…,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0
2582689,null,null,null,null,214676.0,189291.17,0.0,0.0,0.0,10.0,7.0,2017.0,2008.0,22544.6,null,null,22544.6,0.0,null,null,2.0,11.0,6.0,null,0.0,null,19.0,0.0,0.0,0.0,0.0,null,0.0,0.0,27.858,0.0,…,0,1,0,1,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,0
106337,null,39.57,null,null,null,141517.9,0.0,0.0,null,null,12.0,2018.0,null,null,null,null,7729.0,null,43.0,null,1.0,3.0,null,36.0,null,35.0,0.0,null,0.0,null,null,141517.9,null,0.0,0.0,null,…,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,1,0,1,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,0
2647,null,null,null,null,null,108895.164,0.0,22.0,null,null,12.0,2018.0,null,null,null,null,4866.618,null,null,null,2.0,6.0,null,35.0,null,20.0,25.0,null,0.0,null,null,83395.72,null,0.0,54.608,null,…,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,0
1367697,null,null,null,null,0.0,0.0,0.0,0.0,null,null,7.0,2017.0,null,0.0,null,null,0.0,null,null,null,1.0,4.0,null,null,null,null,0.0,null,0.0,null,null,null,null,0.0,0.0,null,…,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1268032,null,null,null,null,37598.0,21033.717,0.0,0.0,null,null,3.0,2017.0,null,1967.6,null,null,1967.6,null,null,null,1.0,6.0,null,null,null,null,1.0,null,0.0,null,null,null,null,0.0,3494.4001,null,…,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1